In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

import dataset
assert tf.config.list_physical_devices('GPU')

Using TensorFlow backend.


In [2]:
MAXLEN = 64
BATCH_SIZE = 32
HIDE_LETTERS = False

def build_model():
    EMBED_DIM = 512
    UNITS = 256

    LETTERS_SIZE = len(dataset.letters_table)
    NIQQUD_SIZE = len(dataset.niqqud_table)
    DAGESH_SIZE = len(dataset.dagesh_table)
    SIN_SIZE = len(dataset.sin_table)

    common_input = tf.keras.Input(batch_shape=(None, MAXLEN), batch_size=BATCH_SIZE)
    
    common = layers.Embedding(LETTERS_SIZE, EMBED_DIM, mask_zero=True)(common_input)
    common = layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(common)
    common = layers.add([common,
                         layers.Bidirectional(layers.LSTM(UNITS, return_sequences=True, dropout=0.1), merge_mode='sum')(common)])

    outputs = [
        layers.Softmax(name='N')(layers.Dense(NIQQUD_SIZE)(common)),
        layers.Softmax(name='D')(layers.Dense(DAGESH_SIZE)(common)),
        layers.Softmax(name='S')(layers.Dense(SIN_SIZE)(common))
    ]
    if HIDE_LETTERS:
        outputs.append(layers.Softmax(name='C')(layers.Dense(LETTERS_SIZE)(common)))
        
    model = tf.keras.Model(inputs=[common_input], outputs=outputs)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    tf.keras.utils.plot_model(model, to_file='model.png')
    model.summary()
    return model
    
model = build_model()
model.save_weights('./checkpoint_uninit')

def add_noise(original):
    noised = np.copy(original)
    indices = np.random.choice(np.arange(noised.size), replace=False, size=int(noised.size / MAXLEN))
    indices = np.unravel_index(indices, noised.shape)
    noised[indices] = 1  # Cannot be 0 since masking means it will not be considered as loss
    return noised

def fit(data, learning_rates):
    x  = add_noise(data.normalized_texts)
    vx = add_noise(data.normalized_validation)
    y  = {'N': data.niqqud_texts,      'D': data.dagesh_texts,       'S': data.sin_texts,      'C': data.normalized_texts     }
    vy = {'N': data.niqqud_validation, 'D':  data.dagesh_validation, 'S': data.sin_validation, 'C': data.normalized_validation}
    if HIDE_LETTERS:
        x  = add_noise(x)
        vx = add_noise(vx)
        y['C'] = data.normalized_texts
        vy['C'] = data.normalized_validation
    return model.fit(x, y, validation_data=(vx, vy), batch_size=BATCH_SIZE, epochs=len(learning_rates),
          callbacks=[
              tf.keras.callbacks.LearningRateScheduler(lambda epoch, lr: learning_rates[epoch], verbose=0),
              # tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints/ckpt_{epoch}', save_weights_only=True),
          ]
    )

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 64, 512)      22528       input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 64, 256)      1574912     embedding[0][0]                  
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 64, 256)      1050624     bidirectional[0][0]              
________

In [3]:
def load_data(source):
    filenames = [os.path.join('texts', f) for f in source]
    return dataset.load_file(filenames, BATCH_SIZE, 0.1, maxlen=MAXLEN, shuffle=True)

In [4]:
rabanit = ['birkat_hamazon.txt', 'kuzari.txt', 'hakdama_leorot.txt', 'hartzaat_harav.txt', 'orhot_hayim.txt', 'rambam_mamre.txt', 'short_table.txt',
           'tomer_dvora.txt', 'elef_layla.txt', 'bialik', 'breslev.txt']

data_rabanit = load_data(rabanit)

In [5]:
modern = ['itzhak_berkman', 'zevi_scharfstein', 'pesah_kaplan', 'abraham_regelson', 'elisha_porat', 'uriel_ofek', 'yisrael_dushman', 'zvi_zviri',
          'sipurim.txt' ,'atar_hashabat.txt', 'ali_baba.txt', 'ricky.txt', 'imagination.txt', 'adamtsair.txt', 'katarsis.txt']

data_modern = load_data(modern)

In [ ]:
model.load_weights('./checkpoint_uninit')
history = fit(data_rabanit, [3e-3, 3e-4])
model.save_weights('./checkpoint_rabanit')

Train on 105753 samples, validate on 11751 samples
Epoch 1/2
 13344/105753 [==>...........................] - ETA: 5:55 - loss: 0.7371 - N_loss: 0.5764 - D_loss: 0.1327 - S_loss: 0.0280 - N_accuracy: 0.7959 - D_accuracy: 0.9481 - S_accuracy: 0.9919

In [ ]:
model.load_weights('./checkpoint_rabanit')
history = fit(data_modern, [3e-3, 3e-4])

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=2)

for n, v in enumerate(['accuracy', 'loss'], 0):
    for n1, t in enumerate(['D', 'N'], 0):
        p = ax[n][n1]
        p.plot(history.history[t + '_' + v][0:])
        p.plot(history.history['val_' + t + '_' +  v][0:])
        p.legend([t + '_Train', t + '_Test'], loc='center right')

plt.tight_layout()

In [ ]:
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, '.')

In [ ]:
def print_predictions(data, k):
    s = slice(k*BATCH_SIZE, (k+1)*BATCH_SIZE)
    batch = data.normalized_validation[s]
    prediction = model.predict(batch)
    [actual_niqqud, actual_dagesh, actual_sin] = [dataset.from_categorical(prediction[0]), dataset.from_categorical(prediction[1]), dataset.from_categorical(prediction[2])]
    [expected_niqqud, expected_dagesh, expected_sin] = [data.niqqud_validation[s], data.dagesh_validation[s], data.sin_validation[s]]
    actual = data.merge(batch, ns=actual_niqqud, ds=actual_dagesh, ss=actual_sin)
    expected = data.merge(batch, ns=expected_niqqud, ds=expected_dagesh, ss=expected_sin)
    for i, (a, e) in enumerate(zip(actual, expected)):
        print('מצוי: ', a)
        print('רצוי: ', e)
        print()

print_predictions(data_modern, 1)